In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Input, Dense, GlobalAveragePooling2D, merge, Dropout, Convolution2D, Lambda
from keras.layers.core import K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

from keras.applications import vgg16

K._LEARNING_PHASE = tf.constant(0)  # test mode

Using TensorFlow backend.


In [2]:
import preprocess2 as pp
# a képek betöltése és előfeldolgozása
csv_data, author_stat = pp.csv_load()
# a 10 legtöbb képpel rendelkező alkotót kiválogatjuk
authors_to_select = author_stat[-10:]
authors_to_select

[('96e7b1bc8d52e18caf0af34fec2e9bcb', 402),
 ('3cc9a44380296d93e68b71a27643c25f', 413)]

In [3]:
csv_data = pp.csv_select(csv_data, authors_to_select)
# az útvonalnak az összes képet tartalmazó könyvtárra kell mutatnia, mert különben
# nem fog tudni belőle elég képet kiolvasni
train_images, labels = pp.load_images(csv_data, "train_sample")

374 images in the result


In [4]:
# adatok megfelelő formátumra hozása a keras számára
encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

[1 1 0 1 0 1 0 0 0 1 0 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1
 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1
 0 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 0 0 0 1 0 1
 0 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0
 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 0 0 0
 1 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1 0 0 1
 0 1 1 1 1 0 0 0 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 1
 1 1 1 0 1 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0
 0 0 1 1]
[[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.

In [5]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [6]:
dream = Input(batch_shape=(None,) + (256,256,3)) # <-- mágia

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=dream)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót

for layer in base_model.layers:
    print(layer.name)

In [7]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
desired_layers = [28, 44, 60, 70, 92, 114, 136, 158, 172, 194]
#desired_layers = [11,12,13]
style_layers = [None]*len(desired_layers)

for i in range(len(desired_layers)):
    style_layers[i] = base_model.layers[desired_layers[i]].output
    style_layers[i] = GlobalAveragePooling2D()(style_layers[i])
    style_layers[i] = Dense(base_model.layers[desired_layers[i]].output_shape[3], activation='relu', name=('style_' + str(i)))(style_layers[i])

In [8]:
# egymás mellé tesszük a különböző szintű feature-öket
style_output = merge(style_layers, mode='concat', name='style_output')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(1024, activation='relu')(style_output)
ff = Dropout(0.5)(ff)
ff = Dense(512, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

In [9]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=1, validation_split=0.2, callbacks=[history])

Train on 299 samples, validate on 75 samples
Epoch 1/1
299/299 [==============================] - 130s - loss: 0.6290 - acc: 0.8763 - val_loss: 3.1807e-05 - val_acc: 1.0000


In [11]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(model.layers):
   print(i, layer.name)

Az Inception V3 konvolúciós rétegei:
0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 ba

# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(train_images, labels_onehot,  batch_size=16, nb_epoch=1, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

In [12]:
predicted_values = model.predict(np.asarray(train_images))

In [13]:
print(predicted_values)

[[  2.81625853e-20   1.00000000e+00]
 [  4.72259287e-20   1.00000000e+00]
 [  1.00000000e+00   5.83125903e-08]
 [  7.02146990e-19   1.00000000e+00]
 [  1.00000000e+00   1.60553348e-10]
 [  9.87109036e-18   1.00000000e+00]
 [  1.00000000e+00   2.72766337e-10]
 [  1.00000000e+00   1.03874625e-13]
 [  1.00000000e+00   1.41525484e-14]
 [  1.67870072e-16   1.00000000e+00]
 [  1.00000000e+00   8.40730471e-11]
 [  1.00000000e+00   3.24361018e-08]
 [  9.10581925e-19   1.00000000e+00]
 [  1.16773233e-15   1.00000000e+00]
 [  1.10504415e-20   1.00000000e+00]
 [  1.00000000e+00   6.59075849e-10]
 [  2.51361550e-23   1.00000000e+00]
 [  1.00000000e+00   3.63854086e-10]
 [  2.37725872e-22   1.00000000e+00]
 [  2.29337190e-18   1.00000000e+00]
 [  1.43621798e-18   1.00000000e+00]
 [  5.10049095e-16   1.00000000e+00]
 [  1.00000000e+00   1.94431271e-09]
 [  1.00000000e+00   9.92261207e-09]
 [  1.00000000e+00   2.59183857e-08]
 [  2.07739711e-18   1.00000000e+00]
 [  9.99991179e-01   8.84697965e-06]
 

In [14]:
style_out = Model(input=base_model.input, output=style_output)
styles = style_out.predict(np.asarray(train_images))
style_dict = {}
for i in range(len(labels)):
    if(labels[i] in style_dict):
        style_dict[labels[i]] = (style_dict[labels[i]] + styles[i]) * 0.5
    else:
        style_dict[labels[i]] = styles[i]
    
print(style_dict)

{'96e7b1bc8d52e18caf0af34fec2e9bcb': array([  2.02929331e-38,   8.31954598e-01,   4.53505874e-01, ...,
         0.00000000e+00,   1.54705465e-01,   4.17272560e-03], dtype=float32), '3cc9a44380296d93e68b71a27643c25f': array([  6.81001306e-01,   4.52478230e-02,   1.56288428e-04, ...,
         2.11013481e-03,   2.17397100e-09,   0.00000000e+00], dtype=float32)}


In [15]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
festok

[1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,


In [16]:
encoder.inverse_transform(festok)

array(['96e7b1bc8d52e18caf0af34fec2e9bcb',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '3cc9a44380296d93e68b71a27643c25f',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '3cc9a44380296d93e68b71a27643c25f',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '3cc9a44380296d93e68b71a27643c25f',
       '3cc9a44380296d93e68b71a27643c25f',
       '3cc9a44380296d93e68b71a27643c25f',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '3cc9a44380296d93e68b71a27643c25f',
       '3cc9a44380296d93e68b71a27643c25f',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '3cc9a44380296d93e68b71a27643c25f',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '3cc9a44380296d93e68b71a27643c25f',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '96e7b1bc8d52e18caf0af34fec2e9bcb',
       '3cc9a44380296d93e68b71a27643c25f',
       '3cc

ones = np.ones((256,256,3))

ones_input = Input(shape =(256,256,3))
synth_layer = Convolution2D(3, 256, 256, init='glorot_uniform', weights=None, border_mode='same', subsample=(1, 1), bias=True)(ones_input)
#style_loss = style_out(synth_layer)

In [17]:
# dimensions of the generated picture.
img_width = 256
img_height = 256

# path to the model weights file.
weights_path = 'vgg16_weights.h5'

# some settings we found interesting
saved_settings = {
    'bad_trip': {'features': {'block4_conv1': 0.05,
                              'block4_conv2': 0.01,
                              'block4_conv3': 0.01},
                 'continuity': 0.1,
                 'dream_l2': 0.8,
                 'jitter': 5},
    'dreamy': {'features': {'convolution2d_10': 0.01,
                            'convolution2d_20': 0.02,
                           'convolution2d_30': 0.05,
                           'convolution2d_40': 0.04},
               'continuity': 0.1,
               'dream_l2': 0.02,
               'jitter': 0},
    'styles': {'features': {'style_0': 0.01,
                            'style_1': 0.02,
                            'style_2': 0.03,
                            'style_3': 0.04,
                            'style_4': 0.05,
                            'style_5': 0.05,
                            'style_6': 0.04,
                            'style_7': 0.03,
                            'style_8': 0.02,
                            'style_9': 0.01},
               'continuity': 0.1,
               'dream_l2': 0.02,
               'jitter': 0}
}

In [18]:
def random_image():
    img = np.random.random((256,256,3))
    return img

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image
def deprocess_image(x):
    x = x.reshape((img_width, img_height, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

img_size = (img_width, img_height, 3)
# this will contain our generated image
#dream = Input(batch_shape=(1,) + img_size)

# build the VGG16 network with our placeholder
# the model will be loaded with pre-trained ImageNet weights
model2 = vgg16.VGG16(input_tensor=dream, weights='imagenet', include_top=False)
print('Model loaded.')

In [19]:
# the settings we will use in this experiment
settings = saved_settings['styles']


# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [20]:
# continuity loss util function
def continuity_loss(x):
    assert K.ndim(x) == 4
    if K.image_dim_ordering() == 'th':
        a = K.square(x[:, :, :img_width - 1, :img_height - 1] -
                     x[:, :, 1:, :img_height - 1])
        b = K.square(x[:, :, :img_width - 1, :img_height - 1] -
                     x[:, :, :img_width - 1, 1:])
    else:
        a = K.square(x[:, :img_width - 1, :img_height-1, :] -
                     x[:, 1:, :img_height - 1, :])
        b = K.square(x[:, :img_width - 1, :img_height-1, :] -
                     x[:, :img_width - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

# define the loss
loss = K.variable(0.)
for layer_name in settings['features']:
    # add the L2 norm of the features of a layer to the loss
    assert layer_name in layer_dict.keys(), 'Layer ' + layer_name + ' not found in model.'
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    print(layer_dict[layer_name])
    print(x)
    shape = layer_dict[layer_name].output_shape
    print(shape)
    # we avoid border artifacts by only involving non-border pixels in the loss
    loss -= coeff * K.sum(K.square(x[:,2: shape[1] - 2, 2: shape[2] - 2, :])) / np.prod(shape[1:])

# add continuity loss (gives image local coherence, can result in an artful blur)
loss += settings['continuity'] * continuity_loss(dream) / np.prod(img_size)
# add image L2 norm to loss (prevents pixels from taking very high values, makes image darker)
loss += settings['dream_l2'] * K.sum(K.square(dream)) / np.prod(img_size)

# feel free to further modify the loss as you see fit, to achieve new effects...

# define the loss
loss = K.variable(0.)
for layer_name in settings['features']:
    # add the L2 norm of the features of a layer to the loss
    assert layer_name in layer_dict.keys(), 'Layer ' + layer_name + ' not found in model.'
    coeff = settings['features'][layer_name]
    x = layer_dict[layer_name].output
    print(layer_dict[layer_name])
    print(x)
    shape = layer_dict[layer_name].output_shape
    print(shape)
    # we avoid border artifacts by only involving non-border pixels in the loss
    loss -= coeff * K.sum(K.square(x[:,2: shape[1] - 2])) / np.prod(shape[1:])
    
# add continuity loss (gives image local coherence, can result in an artful blur)
loss += settings['continuity'] * continuity_loss(dream) / np.prod(img_size)
# add image L2 norm to loss (prevents pixels from taking very high values, makes image darker)
loss += settings['dream_l2'] * K.sum(K.square(dream)) / np.prod(img_size)

# feel free to further modify the loss as you see fit, to achieve new effects...

In [26]:
# define the loss
loss = K.variable(0.)
# most az első feső stílusát akarjuk
loss += K.sum(K.square(model.output - [0,1]))
print(loss)
loss += K.sum(K.square(style_output - style_dict['96e7b1bc8d52e18caf0af34fec2e9bcb']))
print(loss)
# add continuity loss (gives image local coherence, can result in an artful blur)
#loss += settings['continuity'] * continuity_loss(dream) / np.prod(img_size)
# add image L2 norm to loss (prevents pixels from taking very high values, makes image darker)
#loss += settings['dream_l2'] * K.sum(K.square(dream)) / np.prod(img_size)

# feel free to further modify the loss as you see fit, to achieve new effects...

Tensor("add_188:0", shape=(), dtype=float32)
Tensor("add_189:0", shape=(), dtype=float32)


In [31]:
# compute the gradients of the dream wrt the loss
grads = K.gradients(loss, dream)
print(loss)
outputs = [loss]
if type(grads) in {list, tuple}:
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([dream, K.learning_phase()], outputs)
def eval_loss_and_grads(x):
    x = x.reshape((1,) + img_size)
    outs = f_outputs([x, 1])
    loss_value = outs[0]
    #print(outs[0])
    if len(outs[1:]) == 1:
        grad_values = (outs[1]).flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

Tensor("add_189:0", shape=(), dtype=float32)


In [32]:
# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [35]:
# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the loss
#x = preprocess_image('trump.jpg')
x = random_image()
y = x.copy()
for i in range(5):
    print('Start of iteration', i)
    start_time = time.time()

    # add a random jitter to the initial image. This will be reverted at decoding time
    random_jitter = (settings['jitter'] * 2) * (np.random.random((256,256,3)) - 0.5)
    x += random_jitter

    # run L-BFGS for 7 steps
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=10)
    print('Current loss value:', min_val)
    # decode the dream and save it
    x = x.reshape((256,256,3))
    x -= random_jitter
    img = deprocess_image(np.copy(x))
    fname = 'reslt_at_iteration_%s.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 492.317
Image saved as reslt_at_iteration_0.png
Iteration 0 completed in 9s
Start of iteration 1
Current loss value: 481.716
Image saved as reslt_at_iteration_1.png
Iteration 1 completed in 9s
Start of iteration 2
Current loss value: 480.394
Image saved as reslt_at_iteration_2.png
Iteration 2 completed in 10s
Start of iteration 3
Current loss value: 474.827
Image saved as reslt_at_iteration_3.png
Iteration 3 completed in 10s
Start of iteration 4
Current loss value: 471.246
Image saved as reslt_at_iteration_4.png
Iteration 4 completed in 9s
Start of iteration 5
Current loss value: 470.59
Image saved as reslt_at_iteration_5.png
Iteration 5 completed in 9s
Start of iteration 6
Current loss value: 470.221
Image saved as reslt_at_iteration_6.png
Iteration 6 completed in 9s
Start of iteration 7
Current loss value: 470.172
Image saved as reslt_at_iteration_7.png
Iteration 7 completed in 11s
Start of iteration 8
Current loss value: 470.079
Image saved a

KeyboardInterrupt: 